# 06 CNN-Reg-Dense

In [ ]:
import config
from utils import data_model
from utils.model import Model

import pandas as pd
import numpy as np
import tensorflow as tf

## Load data

In [ ]:
data, imgs_left, imgs_right = data_model.load(
    config.PATH_DATA_FEATURES01_DLIB_AUGMENTED_NORM_CSV,
    config.PATH_DATA_FEATURES01_DLIB_AUGMENTED_NORM_IMGS_LEFT,
    config.PATH_DATA_FEATURES01_DLIB_AUGMENTED_NORM_IMGS_RIGHT
)

In [ ]:
data.head()

## Split data

In [ ]:
(
    (train_data, train_imgs_left, train_imgs_right),
    (validation_data, validation_imgs_left, validation_imgs_right),
    (test_data, test_imgs_left, test_imgs_right)
) = data_model.split(
    data, imgs_left, imgs_right,
    train_size=0.95,
    validation_size=0.95,
    random_state=42
)

In [ ]:
print("Train length: {}".format(len(train_data)))
print("Validation length: {}".format(len(validation_data)))
print("Test length: {}".format(len(test_data)))

## Model

### Architecture

![](img/models/06-CRD.png)

In [ ]:
def get_model(features, left_imgs, right_imgs, keep_prob):
    new_shape = np.array([-1, 20, 30, 1])
    with tf.variable_scope('model'):
        # Left Eye Img
        l_input = tf.reshape(left_imgs, new_shape)
        # 20x30x1
        cnn_l_01 = tf.layers.conv2d(
            inputs=l_input, 
            filters=32, 
            kernel_size=3, 
            strides=1,
            padding="SAME"
        )
        #20x30x32
        cnn_l_02 = tf.layers.conv2d(
            inputs=cnn_l_01, 
            filters=64, 
            kernel_size=3, 
            strides=2,
            padding="VALID"
        )
        # 10x15x64
        
        # Right Eye Img  
        r_input = tf.reshape(right_imgs, new_shape)      
        # 20x30x1
        cnn_r_01 = tf.layers.conv2d(
            inputs=r_input, 
            filters=32, 
            kernel_size=3, 
            strides=1,
            padding="SAME"
        )    
        # 20x30x32
        cnn_r_02 = tf.layers.conv2d(
            inputs=cnn_r_01, 
            filters=64, 
            kernel_size=3, 
            strides=2,
            padding="VALID"
        )
        # 10x15x64
        
        # Flatten convs, concat & dense        
        left_flat = tf.contrib.layers.flatten (cnn_l_02)
        right_flat =  tf.contrib.layers.flatten (cnn_r_02)
        img_concat = tf.concat(
            values=[left_flat, right_flat],
            axis=1
        )
        # 19200
        img_dense01 = tf.layers.dense(
            inputs=img_concat,
            units=4096,
            activation=tf.nn.relu,
        )
        img_dropout01 = tf.nn.dropout(
            x=img_dense01,
            keep_prob=keep_prob
        )        
        img_dense02 = tf.layers.dense(
            inputs=img_dropout01,
            units=1024,
            activation=tf.nn.relu,
        )
        img_dropout02 = tf.nn.dropout(
            x=img_dense02,
            keep_prob=keep_prob
        )       
        img_dense03 = tf.layers.dense(
            inputs=img_dropout02,
            units=512,
            activation=tf.nn.relu,
        )  
        img_dropout03 = tf.nn.dropout(
            x=img_dense03,
            keep_prob=keep_prob
        )       
        img_dense04 = tf.layers.dense(
            inputs=img_dropout03,
            units=256,
            activation=tf.nn.relu,
        )       
        # Concat imgs with features, dense x 2 and output
        global_concat = tf.concat(
            values=[features, img_dense04],
            axis=1
        )
        global_dense01 = tf.layers.dense(global_concat, 128, activation=tf.nn.relu)
        global_dropout01 = tf.nn.dropout(
            x=global_dense01,
            keep_prob=keep_prob
        )
        global_dense02 = tf.layers.dense(global_dropout01, 64, activation=tf.nn.relu)
        global_dense03 = tf.layers.dense(global_dense02, 2, activation=None)
        
        return global_dense03

In [ ]:
print("Parameters to adjust: {}".format(
    (
        (20*30*32) +
        (10*15*64)
    )*2 +
    (10*15*64*2)*4096 +
    (4096*1024) +
    (1024*512) + 
    (512*256) + 
    (256+12)*128 +
    (128*64) +
    (64*2)
))


### Parameters

In [ ]:
MODEL_NAME = 'CRD-02'

EPOCHS = 100
BATCH_SIZE = 256
LEARNING_RATE = 0.0005
KEEP_PROB = 0.85

## Runs

In [ ]:
model = Model(MODEL_NAME, get_model)

### Train

In [ ]:
model.train(
    train_data, train_imgs_left, train_imgs_right,
    validation_data, validation_imgs_left, validation_imgs_right,
    BATCH_SIZE, EPOCHS, LEARNING_RATE, KEEP_PROB
)

### Test

In [ ]:
model_test = Model(MODEL_NAME, saved_model=MODEL_NAME+".final")

In [ ]:
model_test.test(
    test_data, test_imgs_left, test_imgs_right
)

### Results


| Name | Epochs | Batch Size | Learning rate  | Keep_prob | Train | Validation | Test |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| CRD-01 | 10 | 256 | 0.001 | 1.0 | 0.07739225029945374 | 0.1429390162229538 | 0.14581248 |
| CRD-02 | 100 | 256 | 0.0005 | 0.85 | 0.05397592484951019 | 0.1266249716281891 | 0.12931317 |
